In [10]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

# Sütun isimleri
column_names = [
    "unit_number", "time_in_cycles", "op_setting_1", "op_setting_2", "op_setting_3"
] + [f"sensor_measurement_{i}" for i in range(1, 22)]

# Dosyayı oku
df = pd.read_csv("train_FD001.txt", sep=" ", header=None)
df = df.dropna(axis=1, how='all')  # boş sütunları sil
df.columns = column_names

df.head()

# Her motorun en uzun yaşadığı cycle'ı bul
rul_lookup = df.groupby("unit_number")["time_in_cycles"].max()

# Her satır için RUL hesapla: son cycle - o satırdaki cycle
df["RUL"] = df.apply(lambda row: rul_lookup[row["unit_number"]] - row["time_in_cycles"], axis=1)

# Kontrol için ilk birkaç satıra bakalım
df[["unit_number", "time_in_cycles", "RUL"]].head()

from sklearn.preprocessing import MinMaxScaler

# Sabit sensörleri bul ve at
stds = df.iloc[:, 5:-1].std()  # sensör sütunları 5. sütundan başlıyor, sonuncusu RUL
low_variance_cols = stds[stds < 0.01].index.tolist()
df = df.drop(columns=low_variance_cols)

# Kalan sensörleri seç
sensor_cols = [col for col in df.columns if "sensor_measurement" in col]

# Normalize et (0-1 arası)
scaler = MinMaxScaler()
df[sensor_cols] = scaler.fit_transform(df[sensor_cols])

# Kontrol
df[["unit_number", "time_in_cycles"] + sensor_cols[:3] + ["RUL"]].head()



Saving train_FD001.txt to train_FD001 (2).txt


,unit_number,time_in_cycles,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,RUL
0,1,1,0.183735,0.406802,0.309757,191.0
1,1,2,0.283133,0.453019,0.352633,190.0
2,1,3,0.343373,0.369523,0.370527,189.0
3,1,4,0.343373,0.256159,0.331195,188.0
4,1,5,0.349398,0.257467,0.404625,187.0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Giriş (X) ve hedef (y)
X = df[sensor_cols]
y = df["RUL"]

# Eğitim-test verisi ayır (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli tanımla ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Tahmin yap
y_pred = model.predict(X_test)

# Başarı ölç
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"R2 Score: {r2}")


Mean Absolute Error (MAE): 29.520925611824573
R2 Score: 0.6257788713369965


In [12]:
from google.colab import files
uploaded = files.upload()


Saving RUL_FD001.txt to RUL_FD001 (3).txt
Saving test_FD001.txt to test_FD001 (2).txt


In [14]:
# Daha önce kullandığımız sütun isimlerini tekrar tanımlayalım
column_names = [
    "unit_number", "time_in_cycles", "op_setting_1", "op_setting_2", "op_setting_3"
] + [f"sensor_measurement_{i}" for i in range(1, 22)]

# Test verisini oku
test_df = pd.read_csv("test_FD001.txt", sep=" ", header=None)
test_df = test_df.dropna(axis=1, how='all')
test_df.columns = column_names

# Gerçek RUL (kalan ömür) verisini oku
rul_df = pd.read_csv("RUL_FD001.txt", header=None, names=["RUL"])


In [15]:
# Her motorun en son verisini al (motorun en güncel hali)
latest_cycles = test_df.groupby("unit_number").last().reset_index()


In [16]:
# Sabit sensörleri çıkar (aynı temizliği yapıyoruz)
latest_cycles = latest_cycles.drop(columns=low_variance_cols)

# Normalize et (önceden eğittiğimiz scaler'ı kullan)
latest_cycles[sensor_cols] = scaler.transform(latest_cycles[sensor_cols])


In [17]:
# Tahmin yap
predictions = model.predict(latest_cycles[sensor_cols])

# Tahmin vs Gerçek tabloyu hazırla
comparison = pd.DataFrame({
    "unit_number": latest_cycles["unit_number"],
    "Gerçek RUL": rul_df["RUL"],
    "Tahmin Edilen RUL": predictions.round()
})

# İlk birkaç sonucu göster
comparison.head()


,unit_number,Gerçek RUL,Tahmin Edilen RUL
0,1,112,170.0
1,2,98,153.0
2,3,69,58.0
3,4,82,112.0
4,5,91,127.0
